In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
281,application_1529568156751_0077,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
spark.version

u'2.3.0'

In [3]:
year = 2016
month = 11

## Import Data

In [4]:
file_path = "hdfs:///Projects/TrafficFlow/TrafficFlowParquet/TrafficFlowAll/Year=" + str(year)+ "/Month=" + str(month) + "/*.parquet"
df_raw = spark.read.parquet(file_path)

In [5]:
df_raw.printSchema()

root
 |-- Timestamp: timestamp (nullable = true)
 |-- Date: date (nullable = true)
 |-- Day: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Minute: integer (nullable = true)
 |-- Road: string (nullable = true)
 |-- Km_Ref: integer (nullable = true)
 |-- Detector_Number: integer (nullable = true)
 |-- Traffic_Direction: short (nullable = true)
 |-- Flow_In: short (nullable = true)
 |-- Average_Speed: short (nullable = true)
 |-- Density: double (nullable = true)
 |-- Sign_Aid_Det_Comms: short (nullable = true)
 |-- Status: short (nullable = true)
 |-- Legend_Group: short (nullable = true)
 |-- Legend_Sign: short (nullable = true)
 |-- Legend_SubSign: short (nullable = true)
 |-- Protocol_Version: string (nullable = true)

In [6]:
df_raw.show(2)

+-------------------+----------+---+---------+----+------+-----+------+---------------+-----------------+-------+-------------+------------------+------------------+------+------------+-----------+--------------+----------------+
|          Timestamp|      Date|Day|DayOfWeek|Hour|Minute| Road|Km_Ref|Detector_Number|Traffic_Direction|Flow_In|Average_Speed|           Density|Sign_Aid_Det_Comms|Status|Legend_Group|Legend_Sign|Legend_SubSign|Protocol_Version|
+-------------------+----------+---+---------+----+------+-----+------+---------------+-----------------+-------+-------------+------------------+------------------+------+------------+-----------+--------------+----------------+
|2016-11-11 07:19:00|2016-11-11| 11|        5|   6|    19|  E4Z| 71400|              1|               78|     14|           93|  9.03225806451613|                 0|     3|         255|          1|             1|               4|
|2016-11-11 23:52:00|2016-11-11| 11|        5|  22|    52|E265O|  1300|         

In [7]:
#Add node name
@udf(StringType())
def generate_node_id(road, km):
    if km < 10:
        return road + "-000" + str(km)[-3:]
    if km < 100:
        return road + "-00" + str(km)[-3:]
    if km < 1000:
        return road + "-0" + str(km)[-3:]
    return road + "-" + str(km)[:-3] + "" + str(km)[-3:]

df = df_raw.withColumn('node', generate_node_id('Road', 'Km_Ref'))

In [8]:
#Check how many unique sensors are in the dataset
df.select('node').dropDuplicates().count()

861

## Filter out errors/noise

In [9]:
# Remove error codes
df_noise_free = df.where(col('Average_Speed') <= 250)

# Only valid flow
df_noise_free = df_noise_free.where(col('Flow_In') >= 0)
df_noise_free = df_noise_free.where(col('Flow_In') <= 120)

# limit speed to 120 km/h as it is used as the main speed limit on motorways
# and we are only interested in average speed during heavy traffic
df_noise_free = df.where(col('Average_Speed') <= 120)

In [10]:
#Check how many unique sensors are in the dataset
df_noise_free.select('node').dropDuplicates().count()

827

In [11]:
df_avg_speed = df_noise_free.select(hour(col('Timestamp')).alias('hour'), col('Average_Speed')) \
    .groupby(col('hour')).avg('Average_Speed') \
    .sort(col('hour'))
    
df_avg_speed.show(24)

+----+------------------+
|hour|avg(Average_Speed)|
+----+------------------+
|   0| 80.90548804597815|
|   1| 81.04797875979347|
|   2| 81.00774697225984|
|   3| 81.47937050497308|
|   4| 82.07139191460804|
|   5| 81.39414692002534|
|   6| 75.06015491639732|
|   7| 71.46532217664871|
|   8| 71.81333782252806|
|   9|  75.4109117880912|
|  10| 76.34356304641476|
|  11| 76.78323822376927|
|  12| 76.02042489151845|
|  13| 74.91951305958327|
|  14| 73.26775500639778|
|  15|  68.9119831966271|
|  16| 63.81235366934743|
|  17|  68.3766544717883|
|  18| 75.12724527663848|
|  19| 78.59904123635371|
|  20| 79.56545950317044|
|  21|  80.0564800108513|
|  22| 80.54483259323759|
|  23| 80.77500783923341|
+----+------------------+

In [12]:
# Only use data between 7:00 to 8:00 and 14:00 to 17:00
df_day_time = df_noise_free.select(
    hour(col('Timestamp')).alias('hour'), 
    col('node'),
    col('Average_Speed')
    ).where("hour > 6 and hour < 18") \
    .where("hour < 9 or hour > 13")

In [13]:
df_avg_speed = df_day_time.groupby(col('node')).avg('Average_Speed') \
    .select(col('node'), col('avg(Average_Speed)').alias('Average_Speed'))
df_avg_speed.show()

+-----------+------------------+
|       node|     Average_Speed|
+-----------+------------------+
|E20_F-60630|  69.7245800616241|
|E425N-58125|              53.5|
| E18O-37735| 59.29376224689745|
|  E6N-12920| 73.48458372263326|
|E73_E-53135| 67.84747274529236|
|  E75W-2840| 57.14751744963121|
|  E75W-4370| 75.46368264529463|
| E4_E-51430|50.770136211770755|
|  E75O-0750| 43.70943768497205|
|  E4N-28500|  90.6906728477686|
|  E4N-32985| 93.12741493415335|
|  E75W-1670|60.224579225176775|
|  E4N-58480|58.936600276079666|
|  E4N-59195| 66.73835629730254|
| E182N-3285| 69.17095376700738|
|  E75O-5180| 71.54878840869348|
|  E4Z-38235| 74.08047991705138|
| E4_A-64770| 50.68012256861178|
|  E4Z-59205|57.883029141877586|
|  E75O-5675| 72.46219833772822|
+-----------+------------------+
only showing top 20 rows

## Write CSV File

In [14]:
df_avg_speed.coalesce(1).write \
    .option('timestampFormat', 'yyyy-MM-dd HH:mm:ss.SSS') \
    .option('sep', ';') \
    .format('com.databricks.spark.csv') \
    .option('header', 'true') \
    .mode('overwrite') \
    .save('hdfs:///Projects/traffic_reginbald/processed_traffic_data/avg_sensor_speed')